In [23]:
import sqlite3
import pandas as pd
import plotly.express as px
import nbformat
import statsmodels
import plotly.graph_objects as go

# Connect to the SQLite database
connexion = sqlite3.connect('db/vivino.db')
cursor = connexion.cursor()


In [143]:
# 1a - Marketing push - price distribution
query1a = ("""
WITH price_buckets AS (
    SELECT  NTILE(10) OVER (ORDER BY price_euros) AS price_bucket,
            price_euros
    FROM    vintages
    WHERE   bottle_volume_ml = 750
)

SELECT  price_bucket,
        AVG(price_euros) AS avg_price,
        COUNT(*) AS wines_count
FROM    price_buckets
GROUP BY price_bucket  
ORDER BY price_bucket
""")

cursor.execute(query1a)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query1a, connexion)
df



Number of rows: 10
[(1, 50.46808243727597, 186), (2, 91.26720430107531, 186), (3, 131.56037634408605, 186), (4, 181.5922580645162, 186), (5, 241.4904838709678, 186), (6, 328.49162162162173, 185), (7, 446.144, 185), (8, 635.586162162162, 185), (9, 964.2643243243243, 185), (10, 2686.8456216216214, 185)]


,price_bucket,avg_price,wines_count
0,1,50.468082,186
1,2,91.267204,186
2,3,131.560376,186
3,4,181.592258,186
4,5,241.490484,186
5,6,328.491622,185
6,7,446.144000,185
7,8,635.586162,185
8,9,964.264324,185
9,10,2686.845622,185


In [150]:
# 1b - Marketing push - ratings distribution
query1b = ("""
WITH ratings_avg_buckets AS (
    SELECT  NTILE(10) OVER (ORDER BY ratings_average) AS ratings_bucket,
            ratings_average
    FROM    vintages
    WHERE   bottle_volume_ml = 750  and ratings_average > 0
)

SELECT  ratings_bucket,
        MIN(ratings_average) AS avg_ratings,
        MAX(ratings_average) AS avg_ratings,
        AVG(ratings_average) AS avg_ratings,
        COUNT(*) AS wines_count
FROM    ratings_avg_buckets
GROUP BY ratings_bucket  
ORDER BY ratings_bucket
""")

cursor.execute(query1b)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query1b, connexion)
df


Number of rows: 10
[(1, 4.4, 4.4, 4.399999999999989, 140), (2, 4.4, 4.4, 4.399999999999989, 140), (3, 4.4, 4.5, 4.437142857142854, 140), (4, 4.5, 4.5, 4.5, 140), (5, 4.5, 4.5, 4.5, 140), (6, 4.5, 4.5, 4.5, 140), (7, 4.5, 4.6, 4.593525179856127, 139), (8, 4.6, 4.6, 4.600000000000012, 139), (9, 4.6, 4.7, 4.656115107913673, 139), (10, 4.7, 4.9, 4.722302158273366, 139)]


,ratings_bucket,avg_ratings,avg_ratings,avg_ratings,wines_count
0,1,4.4,4.4,4.400000,140
1,2,4.4,4.4,4.400000,140
2,3,4.4,4.5,4.437143,140
3,4,4.5,4.5,4.500000,140
4,5,4.5,4.5,4.500000,140
5,6,4.5,4.5,4.500000,140
6,7,4.5,4.6,4.593525,139
7,8,4.6,4.6,4.600000,139
8,9,4.6,4.7,4.656115,139
9,10,4.7,4.9,4.722302,139


In [149]:
# 1c - Marketing push - ratings count distribution
query1c = ("""
WITH ratings_count_buckets AS (
    SELECT  NTILE(10) OVER (ORDER BY ratings_count) AS ratings_count_bucket,
            ratings_count
    FROM    vintages
    WHERE   bottle_volume_ml = 750
)

SELECT  ratings_count_bucket,
        MIN(ratings_count) AS ratings_count,
        MAX(ratings_count) AS ratings_count,
        AVG(ratings_count) AS ratings_count,
        COUNT(*) AS wines_count
FROM    ratings_count_buckets
        
GROUP BY ratings_count_bucket  
ORDER BY ratings_count_bucket
""")

cursor.execute(query1c)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query1c, connexion)
df


Number of rows: 10
[(1, 0, 8, 3.629032258064516, 186), (2, 8, 18, 12.688172043010752, 186), (3, 18, 39, 26.704301075268816, 186), (4, 39, 81, 56.36021505376344, 186), (5, 83, 149, 116.70430107526882, 186), (6, 149, 212, 178.71891891891892, 185), (7, 212, 309, 253.30810810810812, 185), (8, 309, 498, 393.64864864864865, 185), (9, 499, 954, 692.4918918918919, 185), (10, 959, 36681, 3299.7837837837837, 185)]


,ratings_count_bucket,ratings_count,ratings_count,ratings_count,wines_count
0,1,0,8,3.629032,186
1,2,8,18,12.688172,186
2,3,18,39,26.704301,186
3,4,39,81,56.360215,186
4,5,83,149,116.704301,186
5,6,149,212,178.718919,185
6,7,212,309,253.308108,185
7,8,309,498,393.648649,185
8,9,499,954,692.491892,185
9,10,959,36681,3299.783784,185


In [180]:
# 1d - Marketing push - final selection - add  countries
query1d = ("""
SELECT  DISTINCT
        vintages.id,
        vintages.name,
        toplists.name AS toplist_name,
        vintage_toplists_rankings.rank,
        vintages.price_euros,
        vintages.ratings_average,
        vintages.ratings_count,
        vintages.price_discount_percentage AS discount_rate
       
           
FROM    vintages
        JOIN vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
        JOIN toplists ON vintage_toplists_rankings.top_list_id = toplists.id
               
WHERE   
        vintages.ratings_count > 200
        AND vintages.ratings_average >= 4.6
        AND vintages.price_euros < 425
        AND vintages.bottle_volume_ml = 750
           
GROUP BY vintages.id
""")

cursor.execute(query1d)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query1d, connexion)
df          

Number of rows: 10
[(1485238, 'Krug Rosé', 'top_lists.EditorialTopList.2017999.name', 6, 349.0, 4.6, 3605, None), (1776981, 'Burmester Tordiz 40 Year Old Tawny Port', "Vivino's 2020 Wine Style Awards: Portuguese Tawny Port", 1, 337.5, 4.6, 474, None), (2674920, 'Ruinart Dom Ruinart Blanc de Blancs Brut Champagne 2010', "Vivino's 2016 Wine Style Awards: French Champagne", 10, 287.0, 4.6, 465, None), (5128110, 'Giacomo Conterno Barolo Cascina Francia 2012', "Vivino's 2019 Wine Style Awards: Italian Barolo", 9, 420.0, 4.6, 559, None), (12776766, 'Cerbaiona Brunello di Montalcino 2012', "Vivino's 2019 Wine Style Awards: Italian Brunello", 8, 276.25, 4.6, 380, None), (20315417, 'Achaval-Ferrer Finca Altamira Malbec 2015', "Vivino's 2020 Wine Style Awards: Argentinian Malbec", 6, 99.95, 4.6, 666, None), (25043149, 'Cardinale Red 2014', "Vivino's 2020 Wine Style Awards: Napa Valley Bordeaux Blend", 9, 399.0, 4.6, 378, None), (85559438, 'Antinori Tignanello 2016', "Vivino's 2020 Wine Style Awa

,id,name,toplist_name,rank,price_euros,ratings_average,ratings_count,discount_rate
0,1485238,Krug Rosé,top_lists.EditorialTopList.2017999.name,6,349.00,4.6,3605,None
1,1776981,Burmester Tordiz 40 Year Old Tawny Port,Vivino's 2020 Wine Style Awards: Portuguese Ta...,1,337.50,4.6,474,None
2,2674920,Ruinart Dom Ruinart Blanc de Blancs Brut Champ...,Vivino's 2016 Wine Style Awards: French Champagne,10,287.00,4.6,465,None
3,5128110,Giacomo Conterno Barolo Cascina Francia 2012,Vivino's 2019 Wine Style Awards: Italian Barolo,9,420.00,4.6,559,None
4,12776766,Cerbaiona Brunello di Montalcino 2012,Vivino's 2019 Wine Style Awards: Italian Brunello,8,276.25,4.6,380,None
5,20315417,Achaval-Ferrer Finca Altamira Malbec 2015,Vivino's 2020 Wine Style Awards: Argentinian M...,6,99.95,4.6,666,None
6,25043149,Cardinale Red 2014,Vivino's 2020 Wine Style Awards: Napa Valley B...,9,399.00,4.6,378,None
7,85559438,Antinori Tignanello 2016,Vivino's 2020 Wine Style Awards: Tuscan Red,10,420.00,4.6,13709,None
8,152069724,Gitana Lupi Rezerva 2017,Bestsellers in Arizona,15,36.55,4.6,1155,None
9,164942599,Krug Grande Cuvée,Vivino's 2020 Wine Style Awards: French Champa...,6,245.00,4.6,28513,None


In [12]:
# 2 - countries Marketing priorities: 

query2a= ("""
SELECT  countries.users_count,
        countries.name, 
        AVG(wines.ratings_average), 
        sum(wines.ratings_count),
        AVG(vintages.price_euros),
        ROUND((SUM(wines.ratings_count)*1.0/countries.users_count), 2) AS ratings_per_user 
        
FROM    wines
        JOIN regions ON wines.region_id = regions.id
        JOIN countries ON regions.country_code = countries.code
        JOIN vintages ON wines.id = vintages.wine_id
GROUP BY countries.name  
ORDER BY countries.users_count DESC, AVG(wines.ratings_average) DESC, AVG(wines.ratings_count) DESC
LIMIT 100
""") 

# for row in cursor.execute(query3):
#     print(row)
cursor.execute(query2a)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query2a, connexion)
df



Number of rows: 17
[(12273684, 'États-Unis', 4.506250000000001, 1528232, 300.36616071428574, 0.12), (5973301, 'France', 4.484049930651893, 9634864, 1025.3709986130377, 1.61), (4270717, 'Italie', 4.478492239467868, 11898245, 476.0170214338511, 2.79), (2549989, 'Allemagne', 4.475000000000001, 16609, 282.01666666666665, 0.01), (2264396, 'Espagne', 4.467175572519083, 1132535, 282.1422900763359, 0.5), (1601799, 'Suisse', 4.35, 1047, 123.42500000000001, 0.0), (1123535, 'Portugal', 4.429032258064517, 71717, 161.94903225806445, 0.06), (1022965, 'Australie', 4.5, 142532, 495.00937500000003, 0.14), (629532, 'Argentine', 4.42857142857143, 341337, 111.37749999999998, 0.54), (326757, 'Chili', 4.468, 363988, 129.70519999999996, 1.11), (269649, 'Afrique du Sud', 4.465625000000001, 80449, 87.73968749999997, 0.3), (228185, 'Roumanie', 4.4, 166, 36.08, 0.0), (150549, 'Israël', 4.5, 894, 37.05, 0.01), (102235, 'Hongrie', 4.4642857142857135, 48542, 106.825, 0.47), (95693, 'Grèce', 4.4, 610, 124.0, 0.01), 

,users_count,name,AVG(wines.ratings_average),sum(wines.ratings_count),AVG(vintages.price_euros),ratings_per_user
0,12273684,États-Unis,4.506250,1528232,300.366161,0.12
1,5973301,France,4.484050,9634864,1025.370999,1.61
2,4270717,Italie,4.478492,11898245,476.017021,2.79
3,2549989,Allemagne,4.475000,16609,282.016667,0.01
4,2264396,Espagne,4.467176,1132535,282.142290,0.50
5,1601799,Suisse,4.350000,1047,123.425000,0.00
6,1123535,Portugal,4.429032,71717,161.949032,0.06
7,1022965,Australie,4.500000,142532,495.009375,0.14
8,629532,Argentine,4.428571,341337,111.377500,0.54
9,326757,Chili,4.468000,363988,129.705200,1.11


In [13]:
# 3 - best winery >> The winery Key and name are wrong in the database, join is useless
query3a = ("""
SELECT  wines.winery_id, 
        wineries.name ,
        AVG(wines.ratings_average), 
        AVG(wines.ratings_count)        
        
FROM    wines
        JOIN wineries ON wines.winery_id = wineries.id
GROUP BY wines.winery_id  
ORDER BY AVG(wines.ratings_average) DESC, AVG(wines.ratings_count) DESC
LIMIT 100
""") 

# for row in cursor.execute(query3):
#     print(row)
cursor.execute(query3a)
result = cursor.fetchall()
print("Number of rows:", len(result))
#print (result)
df = pd.read_sql_query(query3a, connexion)
df


Number of rows: 4


,winery_id,name,AVG(wines.ratings_average),AVG(wines.ratings_count)
0,1651,Tenuta Tignanello 'Solaia',4.5,2472.0
1,75712,Corte di Cama Sforzato di Valtellina,4.5,58.0
2,1652,Tignanello,4.4,741.0
3,1471,Siepi,4.3,4981.0


In [14]:
# 3 - best winery 
query3b = ("""
SELECT  wines.name,
        wines.id,
        vintages.name,
        CASE 
                WHEN INSTR(SUBSTR(vintages.name,1, LENGTH(vintages.name) - 5), wines.name) > 0 THEN
                        SUBSTR(SUBSTR(vintages.name,1, LENGTH(vintages.name) - 5), 1, INSTR(SUBSTR(vintages.name,1, LENGTH(vintages.name) - 5), wines.name) - 1) || 
                        SUBSTR(SUBSTR(vintages.name,1, LENGTH(vintages.name) - 5), INSTR(SUBSTR(vintages.name,1, LENGTH(vintages.name) - 5), wines.name) + LENGTH(wines.name))
                ELSE SUBSTR(vintages.name,1, LENGTH(vintages.name) - 5)
        END AS winery_name,
        wines.winery_id,
        AVG(wines.ratings_average) AS ratings_avg,
        AVG(wines.ratings_count) AS ratings_cnt
        
FROM    wines
        JOIN vintages ON vintages.wine_id = wines.id
       
GROUP BY wines.winery_id  
HAVING ratings_avg >= 4.5 and ratings_cnt> 35000  
ORDER BY AVG(wines.ratings_count)  DESC, AVG(wines.ratings_average) DESC
LIMIT 20
""") 

cursor.execute(query3b)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query3b, connexion)
df



Number of rows: 14
[('Sassicaia', 5078, 'Tenuta San Guido Sassicaia 2005', 'Tenuta San Guido ', 1413, 4.599999999999998, 107646.0), ('Brut Champagne', 86684, 'Dom Pérignon Brut Champagne 2000', 'Dom Pérignon ', 212823, 4.625000000000001, 95089.75), ('60 Sessantanni Old Vines Primitivo di Manduria', 11890, 'San Marzano 60 Sessantanni Old Vines Primitivo di Manduria 2017', 'San Marzano ', 55648, 4.5, 94289.0), ('Special Selection Cabernet Sauvignon', 66294, 'Caymus Special Selection Cabernet Sauvignon 1997', 'Caymus ', 1301, 4.666666666666667, 80138.66666666667), ('Tignanello', 1652, 'Antinori Tignanello 2005', 'Antinori ', 1252, 4.505, 71223.8), ('Ribera del Duero', 77171, 'Alión Ribera del Duero 2019', 'Alión ', 11049, 4.5, 69278.0), ('Opus One', 1911534, 'Opus One Opus One 2012', ' Opus One', 1368, 4.585714285714286, 68446.0), ('Pauillac (Premier Grand Cru Classé)', 1684223, 'Château Mouton Rothschild Pauillac (Premier Grand Cru Classé) 2009', 'Château Mouton Rothschild ', 64252, 4.55

,name,id,name,winery_name,winery_id,ratings_avg,ratings_cnt
0,Sassicaia,5078,Tenuta San Guido Sassicaia 2005,Tenuta San Guido,1413,4.600000,107646.000000
1,Brut Champagne,86684,Dom Pérignon Brut Champagne 2000,Dom Pérignon,212823,4.625000,95089.750000
2,60 Sessantanni Old Vines Primitivo di Manduria,11890,San Marzano 60 Sessantanni Old Vines Primitivo...,San Marzano,55648,4.500000,94289.000000
3,Special Selection Cabernet Sauvignon,66294,Caymus Special Selection Cabernet Sauvignon 1997,Caymus,1301,4.666667,80138.666667
4,Tignanello,1652,Antinori Tignanello 2005,Antinori,1252,4.505000,71223.800000
5,Ribera del Duero,77171,Alión Ribera del Duero 2019,Alión,11049,4.500000,69278.000000
6,Opus One,1911534,Opus One Opus One 2012,Opus One,1368,4.585714,68446.000000
7,Pauillac (Premier Grand Cru Classé),1684223,Château Mouton Rothschild Pauillac (Premier Gr...,Château Mouton Rothschild,64252,4.550000,62429.500000
8,Unico,77137,Vega Sicilia Unico 2012,Vega Sicilia,11050,4.666667,49002.166667
9,Almaviva,4086,Almaviva Almaviva 2014,Almaviva,17605,4.600000,48075.000000


In [126]:
# 4a - identify wine with more than 10 users at least identofued the following taste : coffee, toast, green apple, cream, and citrus
query4a = ("""

SELECT 
    --keywords.name,
    wines.id,
    keywords_wine.keyword_type,
    wines.name,
    keywords_wine.count,
    COUNT(DISTINCT keywords.name) as flavor_count   
                 
FROM    keywords_wine
        JOIN keywords ON keywords.id = keywords_wine.keyword_id
        JOIN wines ON wines.id = keywords_wine.wine_id

WHERE keywords.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus') and keywords_wine.count >=10
GROUP BY wines.id
HAVING COUNT(DISTINCT keywords.name) >= 5
ORDER BY flavor_count DESC

""")

cursor.execute(query4a)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query4a, connexion)
to_csv = df.to_csv('db/taste2.csv', index = False)
df

Number of rows: 19
[(7122486, 'primary', 'Grande Cuvée', 713, 5), (3630450, 'primary', 'Cuvée des Enchanteleurs Brut Champagne', 31, 5), (3102815, 'primary', 'P2 Plénitude Brut Champagne', 161, 5), (1692390, 'primary', 'Sir Winston Churchill Brut Champagne', 282, 5), (1441099, 'primary', 'Blanc des Millénaires', 160, 5), (1192720, 'primary', 'Dom Ruinart Blanc de Blancs Brut Champagne', 226, 5), (1167182, 'primary', 'MV', 10, 5), (1164702, 'primary', 'R.D Extra Brut Champagne (Récemment Dégorgé)', 163, 5), (1153863, 'primary', 'Sauternes', 13, 5), (1136950, 'primary', 'Comtes de Champagne Blanc de Blancs', 587, 5), (1135215, 'primary', 'Sauternes (Premier Grand Cru Classé)', 10, 5), (1127349, 'primary', 'Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne', 240, 5), (1105696, 'primary', "Trebbiano d'Abruzzo", 21, 5), (86684, 'primary', 'Brut Champagne', 2165, 5), (79836, 'primary', 'La Grande Dame Brut Champagne', 244, 5), (79631, 'primary', 'Vintage', 356, 5), (79162, 'primary', 'Belle

,id,keyword_type,name,count,flavor_count
0,7122486,primary,Grande Cuvée,713,5
1,3630450,primary,Cuvée des Enchanteleurs Brut Champagne,31,5
2,3102815,primary,P2 Plénitude Brut Champagne,161,5
3,1692390,primary,Sir Winston Churchill Brut Champagne,282,5
4,1441099,primary,Blanc des Millénaires,160,5
5,1192720,primary,Dom Ruinart Blanc de Blancs Brut Champagne,226,5
6,1167182,primary,MV,10,5
7,1164702,primary,R.D Extra Brut Champagne (Récemment Dégorgé),163,5
8,1153863,primary,Sauternes,13,5
9,1136950,primary,Comtes de Champagne Blanc de Blancs,587,5


In [137]:
# 4b - verification of 4a
query4b = ("""

WITH taste_in_wine AS (
SELECT 
    --keywords.name,
    wines.id,
    keywords_wine.keyword_type,
    wines.name,
    keywords_wine.count,
    COUNT(DISTINCT keywords.name) as flavor_count   
                 
FROM    keywords_wine
        JOIN keywords ON keywords.id = keywords_wine.keyword_id
        JOIN wines ON wines.id = keywords_wine.wine_id

WHERE keywords.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus') and keywords_wine.count >=10
GROUP BY wines.id
HAVING COUNT(DISTINCT keywords.name) >= 5
ORDER BY flavor_count DESC
)
           
SELECT DISTINCT
        taste_in_wine.id,
        taste_in_wine.name,
        keywords.id,
        keywords.name,
        keywords_wine.keyword_id,
        keywords_wine.count
           
FROM   taste_in_wine
        JOIN keywords_wine ON taste_in_wine.id = keywords_wine.wine_id
        JOIN keywords ON keywords.id = keywords_wine.keyword_id
WHERE keywords.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus') 
ORDER BY taste_in_wine.id


""")

cursor.execute(query4b)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query4b, connexion)
to_csv = df.to_csv('db/taste3.csv', index = False)
df

Number of rows: 95
[(18931, 'La Grande Année Brut Champagne', 417, 'toast', 417, 525), (18931, 'La Grande Année Brut Champagne', 117, 'coffee', 117, 32), (18931, 'La Grande Année Brut Champagne', 123, 'cream', 123, 425), (18931, 'La Grande Année Brut Champagne', 195, 'green apple', 195, 175), (18931, 'La Grande Année Brut Champagne', 106, 'citrus', 106, 454), (74304, 'Cristal Brut Champagne (Millésimé)', 417, 'toast', 417, 464), (74304, 'Cristal Brut Champagne (Millésimé)', 117, 'coffee', 117, 26), (74304, 'Cristal Brut Champagne (Millésimé)', 195, 'green apple', 195, 298), (74304, 'Cristal Brut Champagne (Millésimé)', 106, 'citrus', 106, 954), (74304, 'Cristal Brut Champagne (Millésimé)', 123, 'cream', 123, 496), (79162, 'Belle Epoque Brut Champagne', 195, 'green apple', 195, 224), (79162, 'Belle Epoque Brut Champagne', 417, 'toast', 417, 239), (79162, 'Belle Epoque Brut Champagne', 117, 'coffee', 117, 14), (79162, 'Belle Epoque Brut Champagne', 106, 'citrus', 106, 423), (79162, 'Bell

,id,name,id,name,keyword_id,count
0,18931,La Grande Année Brut Champagne,417,toast,417,525
1,18931,La Grande Année Brut Champagne,117,coffee,117,32
2,18931,La Grande Année Brut Champagne,123,cream,123,425
3,18931,La Grande Année Brut Champagne,195,green apple,195,175
4,18931,La Grande Année Brut Champagne,106,citrus,106,454
...,...,...,...,...,...,...
90,7122486,Grande Cuvée,417,toast,417,713
91,7122486,Grande Cuvée,117,coffee,117,50
92,7122486,Grande Cuvée,123,cream,123,439
93,7122486,Grande Cuvée,106,citrus,106,916


In [15]:
# 5a - Best 5 wines per type of grapes - most common grapes world wide
query5a = ("""
SELECT  DISTINCT 
        grapes.name,
        most_used_grapes_per_country.grape_id,
        most_used_grapes_per_country.wines_count AS grapes_count
                
FROM    most_used_grapes_per_country
        JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
ORDER BY grapes_count DESC
LIMIT 3

""")
cursor.execute(query5a)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query5a, connexion)
df


Number of rows: 3
[('Cabernet Sauvignon', 2, 801751), ('Chardonnay', 5, 604208), ('Pinot Noir', 14, 572334)]


,name,grape_id,grapes_count
0,Cabernet Sauvignon,2,801751
1,Chardonnay,5,604208
2,Pinot Noir,14,572334


In [16]:
# 5b - Best 5 wines per type of grapes -  Country list related to  most common grapes world wide
query5b = ("""

WITH most_common_grapes_WW AS (
        SELECT  DISTINCT 
                grapes.name,
                most_used_grapes_per_country.grape_id,
                most_used_grapes_per_country.wines_count AS grapes_count
               
        FROM    most_used_grapes_per_country
                JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
        ORDER BY grapes_count DESC
        LIMIT 3
)          
SELECT   
        most_used_grapes_per_country.grape_id AS grape_id,
        most_used_grapes_per_country.country_code as country_code
                   
FROM    most_used_grapes_per_country
WHERE   grape_id IN (SELECT grape_id FROM most_common_grapes_WW)
GROUP BY grape_id, country_code

           
 

""")
cursor.execute(query5b)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query5b, connexion)
df

Number of rows: 21
[(2, 'ar'), (2, 'au'), (2, 'cl'), (2, 'es'), (2, 'gr'), (2, 'hr'), (2, 'hu'), (2, 'il'), (2, 'md'), (2, 'ro'), (2, 'us'), (2, 'za'), (5, 'ar'), (5, 'au'), (5, 'fr'), (5, 'it'), (5, 'md'), (5, 'us'), (14, 'ch'), (14, 'fr'), (14, 'us')]


,grape_id,country_code
0,2,ar
1,2,au
2,2,cl
3,2,es
4,2,gr
5,2,hr
6,2,hu
7,2,il
8,2,md
9,2,ro


In [17]:
# 5c - Best 5 wines per type of grapes -  region list  for countries related to most common grapes WW
query5c = ("""

WITH countries_most_common_grapes AS (
    WITH most_common_grapes_WW AS (
            SELECT  DISTINCT 
                    grapes.name,
                    most_used_grapes_per_country.grape_id,
                    most_used_grapes_per_country.wines_count AS grapes_count
                
            FROM    most_used_grapes_per_country
                    JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
            ORDER BY grapes_count DESC
            LIMIT 3
    )          
    SELECT   
            most_used_grapes_per_country.grape_id AS grape_id,
            most_used_grapes_per_country.country_code as country_code
                    
    FROM    most_used_grapes_per_country
    WHERE   grape_id IN (SELECT grape_id FROM most_common_grapes_WW)
    GROUP BY grape_id, country_code
)
           
SELECT
    regions.name AS region_name,
    regions.id AS region_id,
    regions.country_code AS region_country_code,
    countries_most_common_grapes.country_code AS prev_country_code,
    countries_most_common_grapes.grape_id AS grape_id
    
    
FROM    
    regions
    JOIN countries_most_common_grapes ON regions.country_code = countries_most_common_grapes.country_code
    
ORDER BY grape_id, region_id, region_country_code

           
        
""")


cursor.execute(query5c)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query5c, connexion)
df

Number of rows: 410
[('South Australia', 7, 'au', 'au', 2), ('California', 24, 'us', 'us', 2), ('Napa Valley', 25, 'us', 'us', 2), ('Sonoma County', 26, 'us', 'us', 2), ('Oakville', 42, 'us', 'us', 2), ('Willamette Valley', 51, 'us', 'us', 2), ('Yamhill County', 55, 'us', 'us', 2), ('Lodi', 65, 'us', 'us', 2), ('Howell Mountain', 85, 'us', 'us', 2), ('Santa Cruz Mountains', 86, 'us', 'us', 2), ('Stags Leap District', 88, 'us', 'us', 2), ('Calistoga', 91, 'us', 'us', 2), ('Russian River Valley', 94, 'us', 'us', 2), ('Los Carneros', 96, 'us', 'us', 2), ('Sta. Rita Hills', 99, 'us', 'us', 2), ('Alexander Valley', 100, 'us', 'us', 2), ('Rutherford', 105, 'us', 'us', 2), ('Walla Walla Valley', 324, 'us', 'us', 2), ('Columbia Valley', 327, 'us', 'us', 2), ('Barossa Valley', 397, 'au', 'au', 2), ('Ribera del Duero', 405, 'es', 'es', 2), ('Cachapoal Valley', 411, 'cl', 'cl', 2), ('Margaret River', 425, 'au', 'au', 2), ('Toro', 430, 'es', 'es', 2), ('Clare Valley', 444, 'au', 'au', 2), ('Mendoz

,region_name,region_id,region_country_code,prev_country_code,grape_id
0,South Australia,7,au,au,2
1,California,24,us,us,2
2,Napa Valley,25,us,us,2
3,Sonoma County,26,us,us,2
4,Oakville,42,us,us,2
...,...,...,...,...,...
405,Champagne Grand Cru 'Ambonnay',4624,fr,fr,14
406,Champagne Grand Cru 'Avize',4625,fr,fr,14
407,Champagne Grand Cru 'Le Mesnil-sur-Oger',4631,fr,fr,14
408,Champagne Grand Cru,4638,fr,fr,14


In [97]:
# 5c - Best 5 wines per type of grapes - all together and extracting the 5 best for each grapes from all region in  the region list
query5d = ("""

WITH countries_most_common_grapes AS (
    WITH most_common_grapes_WW AS (
            SELECT  DISTINCT 
                    grapes.name,
                    most_used_grapes_per_country.grape_id,
                    most_used_grapes_per_country.wines_count AS grapes_count
                
            FROM    most_used_grapes_per_country
                    JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
            ORDER BY grapes_count DESC
            LIMIT 3
    )          
    SELECT   
            most_used_grapes_per_country.grape_id AS grape_id,
            most_used_grapes_per_country.country_code as country_code
                    
    FROM    most_used_grapes_per_country
    WHERE   grape_id IN (SELECT grape_id FROM most_common_grapes_WW)
    GROUP BY grape_id, country_code
),

ranked_wines AS ( 
        SELECT
                countries_most_common_grapes.grape_id AS grape_id,
                ROW_NUMBER() OVER(
                        PARTITION BY countries_most_common_grapes.grape_id 
                        ORDER BY vintages.ratings_average DESC, 
                        vintages.ratings_count DESC
                        ) as rn,
                vintages.name as wintage_name,
                vintages.ratings_average AS wintages_ratings,
                vintages.ratings_count as vintages_ratings_count,
                wines.name as wines_name,
                wines.ratings_average AS wines_ratings,
                wines.ratings_count as wines_ratings_count,
                regions.name AS region_name,
                regions.country_code AS region_country_code,
                countries_most_common_grapes.country_code AS prev_country_code
                --wines.region_id as wines_region_id,
                --regions.id AS region_id,
                
                   
                
        FROM    regions
                JOIN vintages ON wines.id = vintages.wine_id
                JOIN wines ON regions.id = wines.region_id
                JOIN countries  ON regions.country_code = countries.code
                --JOIN countries_most_common_grapes ON regions.country_code = countries_most_common_grapes.country_code
                JOIN countries_most_common_grapes ON countries.code = countries_most_common_grapes.country_code
                
)
           
SELECT * FROM ranked_wines WHERE rn <= 5
        
""")


cursor.execute(query5d)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query5d, connexion)
df

Number of rows: 15
[(2, 1, 'Scarecrow Cabernet Sauvignon 2015', 4.8, 243, 'Cabernet Sauvignon', 4.8, 2941, 'Rutherford', 'us', 'us'), (2, 2, 'Colgin IX Estate Red 2016', 4.8, 131, 'IX Estate Red', 4.7, 2810, 'Napa Valley', 'us', 'us'), (2, 3, 'Harlan Estate Harlan Estate Red 1998', 4.8, 104, 'Harlan Estate Red', 4.7, 7231, 'Napa Valley', 'us', 'us'), (2, 4, 'Artadi Viña El Pison 2018', 4.8, 64, 'Viña El Pison', 4.6, 3027, 'Vino de España', 'es', 'es'), (2, 5, 'Opus One Opus One 2015', 4.7, 4948, 'Opus One', 4.6, 77053, 'Napa Valley', 'us', 'us'), (5, 1, 'Biondi-Santi Brunello di Montalcino Riserva 1955', 4.9, 35, 'Brunello di Montalcino Riserva', 4.6, 3375, 'Brunello di Montalcino', 'it', 'it'), (5, 2, "Château Doisy-Daëne L'Extravagant de Doisy-Daëne Sauternes 2001", 4.9, 32, "L'Extravagant de Doisy-Daëne Sauternes", 4.6, 465, 'Sauternes', 'fr', 'fr'), (5, 3, 'Pétrus Pomerol 1990', 4.8, 1190, 'Pomerol', 4.7, 32157, 'Pomerol', 'fr', 'fr'), (5, 4, 'Salon Le Mesnil Blanc de Blancs (Cuvée

,grape_id,rn,wintage_name,wintages_ratings,vintages_ratings_count,wines_name,wines_ratings,wines_ratings_count,region_name,region_country_code,prev_country_code
0,2,1,Scarecrow Cabernet Sauvignon 2015,4.8,243,Cabernet Sauvignon,4.8,2941,Rutherford,us,us
1,2,2,Colgin IX Estate Red 2016,4.8,131,IX Estate Red,4.7,2810,Napa Valley,us,us
2,2,3,Harlan Estate Harlan Estate Red 1998,4.8,104,Harlan Estate Red,4.7,7231,Napa Valley,us,us
3,2,4,Artadi Viña El Pison 2018,4.8,64,Viña El Pison,4.6,3027,Vino de España,es,es
4,2,5,Opus One Opus One 2015,4.7,4948,Opus One,4.6,77053,Napa Valley,us,us
5,5,1,Biondi-Santi Brunello di Montalcino Riserva 1955,4.9,35,Brunello di Montalcino Riserva,4.6,3375,Brunello di Montalcino,it,it
6,5,2,Château Doisy-Daëne L'Extravagant de Doisy-Daë...,4.9,32,L'Extravagant de Doisy-Daëne Sauternes,4.6,465,Sauternes,fr,fr
7,5,3,Pétrus Pomerol 1990,4.8,1190,Pomerol,4.7,32157,Pomerol,fr,fr
8,5,4,Salon Le Mesnil Blanc de Blancs (Cuvée S) Brut...,4.8,1004,Le Mesnil Blanc de Blancs (Cuvée S) Brut Champ...,4.7,9564,Champagne Grand Cru 'Le Mesnil-sur-Oger',fr,fr
9,5,5,Château Lafite Rothschild Pauillac (Premier Gr...,4.8,1000,Pauillac (Premier Grand Cru Classé),4.6,39283,Pauillac,fr,fr


In [91]:
# 6a - Country rates on wines vizualisation 
query6a= ("""
SELECT  countries.name AS Country,
        countries.wines_count AS wines_nb,
        countries.wineries_count AS wineries_nb,
        AVG(wines.ratings_average)  AS ratings_avg, 
        AVG(wines.ratings_count) AS ratings_count_avg
        
        
FROM    countries
        JOIN regions ON countries.code = regions.country_code
        JOIN wines ON regions.id = wines.region_id

GROUP BY countries.name
ORDER BY AVG(wines.ratings_average) DESC, AVG(wines.ratings_count) DESC

""") 

cursor.execute(query6a)
result = cursor.fetchall()
print("Number of rows:", len(result))
#print (result)
df = pd.read_sql_query(query6a, connexion)
df




fig1 = px.bar(df.sort_values('ratings_avg', ascending=False), 
             x='Country', 
             y='ratings_avg', 
             title='Wines Rating per Country',
             labels={'ratings_avg': '', 'Country': ''},
             color='ratings_avg',
             color_continuous_scale=px.colors.sequential.Magenta)

# Improve the layout
fig1.update_yaxes(range=[4, max(df['ratings_avg'])], dtick=0.5)
fig1.update_layout(
    title={
        'text': "<b>Wines Rating per Country</b>",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 26}
        })

fig1.add_annotation(
    xref='paper',
    yref='paper',
    x=-0.04,
    y=1.15,
    text='<b>Rating</b>',
    showarrow=False,
    font=dict(
        size=14,
        color="black"
    ),
    align='left',
    ax=0,
    ay=0,
    bordercolor='white',
    borderwidth=1,
    borderpad=4
)

fig1.show()


# fig2 = px.scatter(df, x="wineries_nb", y="wines_nb", trendline="ols", 
#                   title='Correlation between Number of Wineries and Wines',
#                   color_discrete_sequence=['blue'], # set the color of the first trace to blue
#                   labels={'wines_nb': 'Wines Numbers'}) # set the name of the first trace
# fig2.show()


# df['ratings_avg_normalized'] = (df['ratings_avg'] - df['ratings_avg'].min()) / (df['ratings_avg'].max() - df['ratings_avg'].min())
# df['ratings_count_avg_normalized'] = (df['ratings_count_avg'] - df['ratings_count_avg'].min()) / (df['ratings_count_avg'].max() - df['ratings_count_avg'].min())
# fig3 = px.scatter(df, x="wineries_nb", y="ratings_count_avg_normalized", trendline="ols", 
#                   title='Correlation between Ratings and Number of Wineries',
#                   color_discrete_sequence=['blue'], # set the color of the first trace to blue
#                   labels={'ratings_count_avg_normalized': ''}) # set the name of the first trace
# fig3.add_annotation(
#     xref='paper',
#     yref='paper',
#     x=-0.06,
#     y=1.1,
#     text='Rating Normalized',
#     showarrow=False,
#     font=dict(
#         size=14,
#         color="black"
#     ),
#     align='left',
#     ax=0,
#     ay=0,
#     bordercolor='white',
#     borderwidth=1,
#     borderpad=4
# )
# fig3.add_trace(
#     go.Scatter(
#         x=df["wineries_nb"],
#         y=df["ratings_count_avg_normalized"],
#         mode='markers',
#         marker=dict(
#             color='lightpink',
#         ),
#         name='Average Ratings counts' # set the name of the second trace
#     )
# )
# fig3.add_trace(
#     go.Scatter(
#         x=df["wineries_nb"],
#         y=df["ratings_avg_normalized"],
#         mode='markers',
#         marker=dict(
#             color='magenta',
#         ),
#         name='Average Ratings' # set the name of the second trace
#     )
# )
# fig3.show()

Number of rows: 17


In [87]:
# 6b - Country rates on vintage visualization
query6a= ("""
SELECT  countries.name AS Country,
        AVG(vintages.ratings_average)  AS ratings_avg, 
        AVG(vintages.ratings_count) AS ratings_count_avg
        
        
        
FROM    countries
        JOIN regions ON countries.code = regions.country_code
        JOIN wines ON regions.id = wines.region_id
        JOIN vintages ON wines.id = vintages.wine_id

GROUP BY countries.name
ORDER BY AVG(wines.ratings_average) DESC, AVG(wines.ratings_count) DESC

""") 


cursor.execute(query6a)
result = cursor.fetchall()
print("Number of rows:", len(result))
#print (result)
df = pd.read_sql_query(query6a, connexion)
df


fig1 = px.bar(df.sort_values('ratings_avg', ascending=False), 
             x='Country', 
             y='ratings_avg', 
             title='Vintages Rating per Country',
             labels={'ratings_avg': '', 'Country': ''},
             color='ratings_avg',
             color_continuous_scale=px.colors.sequential.Magenta)

# Improve the layout
fig1.update_yaxes(range=[0, 5], dtick=0.5)
fig1.update_layout(
    title={
        'text': "<b>Vintages Rating per Country</b>",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 26}
        })

fig1.add_annotation(
    xref='paper',
    yref='paper',
    x=-0.04,
    y=1.15,
    text='<b>Rating</b>',
    showarrow=False,
    font=dict(
        size=14,
        color="black"
    ),
    align='left',
    ax=0,
    ay=0,
    bordercolor='white',
    borderwidth=1,
    borderpad=4
)

fig1.show()

Number of rows: 17


In [107]:
#7 - Cabernet sauvignon recommendation

query5d = ("""

WITH countries_most_common_grapes AS (
    SELECT   
            most_used_grapes_per_country.grape_id AS grape_id,
            most_used_grapes_per_country.country_code as country_code
                    
    FROM    most_used_grapes_per_country
    WHERE   grape_id = 2 
    
),

ranked_vintage AS ( 
        SELECT
                countries_most_common_grapes.grape_id AS grape_id,
                vintages.name as wintage_name,
                vintages.ratings_average AS vintages_ratings,
                vintages.ratings_count as vintages_ratings_count,
                wines.name as wines_name,
                wines.ratings_average AS ratings,
                wines.ratings_count as ratings_count,
                regions.name AS region_name,
                regions.country_code AS region_country_code
                --countries_most_common_grapes.country_code AS prev_country_code
                --wines.region_id as wines_region_id,
                --regions.id AS region_id,
                
                   
                
        FROM    regions
                JOIN vintages ON wines.id = vintages.wine_id
                JOIN wines ON regions.id = wines.region_id
                --JOIN countries  ON regions.country_code = countries.code
                JOIN countries_most_common_grapes ON regions.country_code = countries_most_common_grapes.country_code
)
           
SELECT * FROM ranked_vintage 
ORDER BY vintages_ratings DESC
LIMIT 3
        
""")


cursor.execute(query5d)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query5d, connexion)
df

Number of rows: 3
[(2, 'Harlan Estate Harlan Estate Red 1998', 4.8, 104, 'Harlan Estate Red', 4.7, 7231, 'Napa Valley', 'us'), (2, 'Scarecrow Cabernet Sauvignon 2015', 4.8, 243, 'Cabernet Sauvignon', 4.8, 2941, 'Rutherford', 'us'), (2, 'Colgin IX Estate Red 2016', 4.8, 131, 'IX Estate Red', 4.7, 2810, 'Napa Valley', 'us')]


,grape_id,wintage_name,vintages_ratings,vintages_ratings_count,wines_name,ratings,ratings_count,region_name,region_country_code
0,2,Harlan Estate Harlan Estate Red 1998,4.8,104,Harlan Estate Red,4.7,7231,Napa Valley,us
1,2,Scarecrow Cabernet Sauvignon 2015,4.8,243,Cabernet Sauvignon,4.8,2941,Rutherford,us
2,2,Colgin IX Estate Red 2016,4.8,131,IX Estate Red,4.7,2810,Napa Valley,us
